In [ ]:
n_jo = 8

be = "/Users/kate/craft/data/benchmark"

gr = "/Users/kate/craft/data/grch"

vb = joinpath(be, "HG002_truth/HG002_GRCh38_1_22_v4.2.1_benchmark.vcf.gz")

vc = joinpath(be, "apply_germline_dna_to_genome/call_germline_variant/pass.vcf.gz")

re = joinpath(gr, "GCA_000001405.15_GRCh38_no_alt_plus_hs38d1_analysis_set.fna.gz")

sd = replace(re, "fna.gz" => "sdf")

rt = "/Users/kate/craft/workflow/Fastq.jl/tool/rtg-tools-3.11/rtg"

In [ ]:
# Make vcfeval SDF

if !isdir(sd)

    run(`$rt format -o $sd $re`)

end

In [ ]:
# Rename chromosomes from numbers to strings

ch = joinpath(gr, "chromosome/n_chrn.tsv")

vcn = replace(vc, "pass" => "pass_rename_chromosomes")

run(`bcftools annotate --threads=$n_jo --rename-chrs=$ch --output=$vcn $vc`)

In [ ]:
## run vcfeval

bd = joinpath(be, "HG002_truth/HG002_GRCh38_1_22_v4.2.1_benchmark_noinconsistent.bed")

ou = joinpath(be, "vcfeval")

run(
    `$rt vcfeval --baseline=$vb --bed-regions=$bd --calls=$vcn --template=$sd --output=$ou --threads=$n_jo`,
)

`--baseline` VCF containing baseline truth variants
`--calls` VCF containing called variants
`--template` SDF of reference genome the the `calls` VCF was made against

In [ ]:
## run hap.py in hap.py docker container

ou = joinpath(be, "hap.py")

run(`docker run -it pkrusche/hap.py bash`)

`hap.py $vb $vcn -f $bd -o $ou -V --engine-vcfeval-path $rt --engine-vcfeval-template $sd`

In [ ]:
# Read more about how to interpret vcfeval results [here](https://cdn.rawgit.com/RealTimeGenomics/rtg-tools/master/installer/resources/tools/RTGOperationsManual/rtg_command_reference.html#vcfeval).

pasu = open(joinpath(pab, "summary.txt"))

su = read(pasu)

su